In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

# Data beans audio reviews generation
This notebooks generate synthetic audio reviews based the data beans table `data_beans.customer_review`
- It uses Google `text-to-speech` service

In [ ]:
! pip install google-cloud-texttospeech
! pip3 install --upgrade --user google-cloud-aiplatform

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Imports and parameters

In [ ]:
import random
import logging
import os
import IPython.display


from tqdm import tqdm
from google.cloud import texttospeech
from google.cloud import bigquery

In [ ]:
PROJECT_ID = "${project_id}"
DATASET_ID = "${bigquery_data_beans_curated_dataset}"
LOCATION = "us-central1"
TABLE_ID = "customer_review"
AUDIO_GEN_DIR = "beans_audios"
VOICES = [ 'en-GB-Wavenet-A','en-GB-Wavenet-B','en-GB-Wavenet-C','en-GB-Wavenet-D']

In [ ]:
logging.getLogger().setLevel(logging.INFO)
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_QUOTA_PROJECT"] = PROJECT_ID

## Imports and parameters

In [ ]:
def _get_reviews():
    client = bigquery.Client()
    sql = f"""
    SELECT *
        FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE review_audio_filename is null 
    ORDER BY customer_review_id
    """
    return client.query(sql).to_dataframe()

def _get_processed_ids(directory):
    ids = []
    for file in os.listdir(directory):
        if file.endswith(".mp3"):
            id = int(file.split("_")[1].split(".")[0])
            ids.append(id)
    return ids

def _synth_audio(review):
  tts_client = texttospeech.TextToSpeechClient()
  synthesis_input = texttospeech.SynthesisInput(text=review)
  voice_name = random.choice(VOICES)
  voice = texttospeech.VoiceSelectionParams(language_code="en-GB", name=voice_name)
  audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
  response = tts_client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)
  return response.audio_content

def _save_audio(audio,id):
    with open(os.path.join(AUDIO_GEN_DIR,f"audio_{id}.mp3"), "wb") as f:
              f.write(audio)

## Audio generation

In [ ]:
def gen_audio_reviews():
    os.makedirs(AUDIO_GEN_DIR, exist_ok=True)
    reviews_pd = _get_reviews()
    for _, row in tqdm(reviews_pd.iterrows(), total=len(reviews_pd), desc="Generating Audios"):
        review = row['review_text']
        customer_review_id = row['customer_review_id']
        audio = _synth_audio(review)
        _save_audio(audio,customer_review_id)

In [ ]:
gen_audio_reviews()

In [ ]:
from IPython.display import Audio 
from IPython.core.display import display
processed_ids = _get_processed_ids(AUDIO_GEN_DIR)

for item in processed_ids:
  filename = os.path.join(AUDIO_GEN_DIR,f"audio_{item}.mp3")
  display(Audio(filename, autoplay=True,rate=16000))